# 0目录
1. SQL基础
2. 合并

# 1 SQL 基础
## 1.1读取csv文件

In [1]:
data  t_user_test;
    infile 'data/t_user_test.csv' dlm=',' firstobs = 2;
    input uid 
        age 
        sex 
        active_date $10. 
        limit;
run;
 
proc print data=t_user_test;
    title '用户数据表 t_user_test';
run;

Obs,uid,age,sex,active_date,limit
1,26308,30,1,2016-02-16,5.97468
2,78209,40,1,2016-02-21,5.29215
3,51930,35,1,2016-04-19,6.29205
4,10113,25,1,2016-03-12,6.29205
5,17067,35,1,2016-02-16,5.97468
6,46744,35,1,2015-12-17,5.97468
7,31747,30,2,2016-02-13,6.29205
8,48366,30,2,2016-01-28,5.97468
9,49725,35,1,2016-01-28,5.29215


## 1.2 SQL查询

In [2]:
proc sql;
    title 'SQL 查询';
    select uid, age, sex, active_date, limit as money
        from t_user_test
        where active_date > '2016-03-01'
        order by active_date desc; /* 默认升序asce */
quit;

uid,age,sex,active_date,money
51930,35,1,2016-04-19,6.292055
10113,25,1,2016-03-12,6.292055


## 1.3 group by

In [4]:
proc sql;
    title 'group by';
    select age, sum(limit) as money /*Group by 若不与sum()连用，sas会将其变换为order by*/
        from t_user_test
        group by age
        having money > 5 /*Having限定输出的观测*/
        order by money desc;
quit;

age,money
35,23.53356
30,18.24141
25,6.292055
40,5.292154


# 2 SQL 合并
## 2.1 准备数据

In [29]:
data  t_user;
    infile 'data/t_user.csv' dlm=',' firstobs = 2 obs = 5;
    input uid 
        age 
        sex 
        active_date $10. 
        limit;
run;
 
proc print data=t_user;
    title '用户数据表 t_user';
run;

Obs,uid,age,sex,active_date,limit
1,26308,30,1,2016-02-16,5.97468
2,78209,40,1,2016-02-21,5.29215
3,51930,35,1,2016-04-19,6.29205
4,10113,25,1,2016-03-12,6.29205


In [30]:
data  t_loan_sum;
    infile 'data/t_loan_sum.csv' dlm=',' firstobs = 2;
    input uid 
        month $
        loan_sum;
run;

proc print data=t_loan_sum;
    title '贷款总金额表 t_loan_sum';
run;

Obs,uid,month,loan_sum
1,34939,2016-11,6.31642
2,26308,2016-11,6.21263
3,17067,2016-11,6.15341
4,58005,2016-11,6.79313
5,52453,2016-11,4.29265
6,45117,2016-11,6.77054
7,41592,2016-11,5.50118
8,46744,2016-11,6.15341
9,21745,2016-11,5.40542


## 2.2  合并
### 横向合并

In [31]:
* method 1;
proc sql;
title '横向合并_1 join';
    select u.uid, age, sex, active_date, limit, month, loan_sum 
        from t_user as u
        inner join t_loan_sum as l
        on t_user.uid = t_loan_sum.uid
        order by uid;
quit;

uid,age,sex,active_date,limit,month,loan_sum
26308,30,1,2016-02-16,5.974677,2016-11,6.212631


In [32]:
* method 2;
proc sql;
title '横向合并_2 join';
    select u.uid, age, sex, active_date, limit, month, loan_sum 
        from t_user as u, t_loan_sum as l
        where t_user.uid = t_loan_sum.uid
        order by uid;
quit;

uid,age,sex,active_date,limit,month,loan_sum
26308,30,1,2016-02-16,5.974677,2016-11,6.212631


how about merge?
用in来控制非交集行

In [33]:
* method 3;
proc sort data = t_user;
    by uid;
run;

proc sort data = t_loan_sum;
    by uid;
run;

data merge1;
    merge t_user(in = x) t_loan_sum(in = y);
    by uid;
    if x=1 and y=1; /*用if 非where*/
run;

proc print data = merge1;
    title '横向合并_3 merge';
run;

Obs,uid,age,sex,active_date,limit,month,loan_sum
1,26308,30,1,2016-02-16,5.97468,2016-11,6.21263


In [36]:
data merge2;
    merge t_user t_loan_sum;
run;

proc print data = merge2;
    title '横向合并_4 merge, 普通merge没有选择交集行，若两个表存在相同字段uid，后表会覆盖前表的uid';
run;

Obs,uid,age,sex,active_date,limit,month,loan_sum
1,17067,25,1,2016-03-12,6.29205,2016-11,6.15341
2,21745,30,1,2016-02-16,5.97468,2016-11,5.40542
3,26308,35,1,2016-04-19,6.29205,2016-11,6.21263
4,34939,40,1,2016-02-21,5.29215,2016-11,6.31642
5,41592,.,.,,.,2016-11,5.50118
6,45117,.,.,,.,2016-11,6.77054
7,46744,.,.,,.,2016-11,6.15341
8,52453,.,.,,.,2016-11,4.29265
9,58005,.,.,,.,2016-11,6.79313


In [37]:
* method 5;
data setset;
    set t_user; 
    set t_loan_sum;
run;

proc print data = setset;
    title '横向合并_5 setset，但若两个表存在相同字段uid，后表会覆盖前表的uid，是merge缩减版';
run;

Obs,uid,age,sex,active_date,limit,month,loan_sum
1,17067,25,1,2016-03-12,6.29205,2016-11,6.15341
2,21745,30,1,2016-02-16,5.97468,2016-11,5.40542
3,26308,35,1,2016-04-19,6.29205,2016-11,6.21263
4,34939,40,1,2016-02-21,5.29215,2016-11,6.31642


### 纵向合并

In [40]:
proc sql;
    title '纵向合并 union';
    select * from t_user_test
        union 
        select * from t_user
        where active_date > '2016-03-01'
        order by active_date desc; /* 默认升序asce */
quit;

uid,age,sex,active_date,limit
51930,35,1,2016-04-19,6.292055
10113,25,1,2016-03-12,6.292055
78209,40,1,2016-02-21,5.292154
26308,30,1,2016-02-16,5.974677
17067,35,1,2016-02-16,5.974677
31747,30,2,2016-02-13,6.292055
48366,30,2,2016-01-28,5.974677
49725,35,1,2016-01-28,5.292154
46744,35,1,2015-12-17,5.974677


how about set
1. 双set，后表覆盖前表
2. 单set，前执行前表，在执行后表，不覆盖

In [41]:
data set1;
    set t_user_test;
    set t_user;
run;

proc print data = set1;
    title '双set，后表覆盖前表';
run;

Obs,uid,age,sex,active_date,limit
1,10113,25,1,2016-03-12,6.29205
2,26308,30,1,2016-02-16,5.97468
3,51930,35,1,2016-04-19,6.29205
4,78209,40,1,2016-02-21,5.29215


In [42]:
data set1;
    set t_user_test t_user;
run;

proc print data = set1;
    title '单set，前执行前表，在执行后表，不覆盖';
run;

Obs,uid,age,sex,active_date,limit
1,26308,30,1,2016-02-16,5.97468
2,78209,40,1,2016-02-21,5.29215
3,51930,35,1,2016-04-19,6.29205
4,10113,25,1,2016-03-12,6.29205
5,17067,35,1,2016-02-16,5.97468
6,46744,35,1,2015-12-17,5.97468
7,31747,30,2,2016-02-13,6.29205
8,48366,30,2,2016-01-28,5.97468
9,49725,35,1,2016-01-28,5.29215
10,10113,25,1,2016-03-12,6.29205
